# **구글 드라이브 Mount**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **패키지 설치 및 Import**

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6bwicxhn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6bwicxhn


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

ConnectionError: ignored

# **데이터 전처리 과정** (Augmentation)


In [ ]:
import pandas as pd

data = pd.read_excel('./한국어_단발성_대화_데이터셋.xlsx') #경로 설정

# 문장내용과 감정 Column 만 추출
data = data[['Sentence','Emotion']]

# 강한 긍정 : 행복 (0)
# 약한 긍정 : 중립 (1)
# 약한 부정 : 공포, 놀람, 슬픔 (2)
# 강한 부정 : 혐오, 분노 (3)

data.loc[data.Emotion=='행복','Emotion'] = 0
data.loc[data.Emotion=='중립','Emotion'] = 1
data.loc[data.Emotion=='공포','Emotion'] = 2
data.loc[data.Emotion=='놀람','Emotion'] = 2
data.loc[data.Emotion=='슬픔','Emotion'] = 2
data.loc[data.Emotion=='분노','Emotion'] = 3
data.loc[data.Emotion=='혐오','Emotion'] = 3


pos_strong = data.loc[data['Emotion'] == 0]
pos_week = data.loc[data['Emotion'] == 1]
neg_strong = data.loc[data['Emotion'] == 3]
neg_week = data.loc[data['Emotion'] == 2]


13306
# Train : Validation = 8 : 2
pos_strong_train =  pos_strong.iloc[:4831]
strong_copy = pos_strong_train.copy()

strong_sub = pos_week.copy()
strong_sub.loc[strong_sub.Emotion==1,'Emotion'] = 0
strong_sub = strong_sub.iloc[:3644]

pos_strong_train = pd.concat([pos_strong_train,strong_copy,strong_sub])

pos_strong_test = pos_strong.iloc[4831:]


pos_week_train =  pos_week.iloc[:3865]
week_copy = pos_week_train.copy()

week_sub = pos_strong.copy()
week_sub.loc[week_sub.Emotion==0,'Emotion'] = 1
week_sub = week_sub.iloc[:-461]

pos_week_train = pd.concat([pos_week_train,week_copy,week_sub])

pos_week_test = pos_week.iloc[3865:]



neg_strong_train =  neg_strong.iloc[:8875]
neg_strong_copy = neg_strong.iloc[:4431]
neg_strong_train = pd.concat([neg_strong_train,neg_strong_copy])

neg_strong_test = neg_strong.iloc[8875:]



neg_week_train =  neg_week.iloc[:13306]
neg_week_test = neg_week.iloc[13306:]

# Concatenate
data_train = pd.concat([pos_strong_train,pos_week_train,neg_strong_train,neg_week_train])
data_test = pd.concat([pos_strong_test,pos_week_test,neg_strong_test,neg_week_test])

#Shuffle
dataset_train = data_train.sample(frac=1)
dataset_test = data_test.sample(frac=1)

dataset_train.to_excel('./drive/MyDrive/21-2/data_train.xlsx', index = False) 
dataset_test.to_excel('./drive/MyDrive/21-2/data_test.xlsx', index = False)

a = data_train['Emotion'].to_list()

print("강한 긍정 : ",a.count(0))
print("약한 긍정 : ",a.count(1))
print("약한 부정 : ",a.count(2))
print("강한 부정 : ",a.count(3))

data_train = []
for q, label in zip(dataset_train['Sentence'], dataset_train['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_train.append(data)

data_test = []
for q, label in zip(dataset_test['Sentence'], dataset_test['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_test.append(data)


강한 긍정 :  13306
약한 긍정 :  13306
약한 부정 :  13306
강한 부정 :  13306


# **데이터 전처리 과정** (비율 Cut)



In [ ]:
import pandas as pd

data = pd.read_excel('./한국어_단발성_대화_데이터셋.xlsx') #경로 설정

# 문장내용과 감정 Column 만 추출
data = data[['Sentence','Emotion']]

# 강한 긍정 : 행복 (0)
# 약한 긍정 : 중립 (1)
# 약한 부정 : 공포, 놀람, 슬픔 (2)
# 강한 부정 : 혐오, 분노 (3)

data.loc[data.Emotion=='행복','Emotion'] = 0
data.loc[data.Emotion=='중립','Emotion'] = 1
data.loc[data.Emotion=='공포','Emotion'] = 2
data.loc[data.Emotion=='놀람','Emotion'] = 2
data.loc[data.Emotion=='슬픔','Emotion'] = 2
data.loc[data.Emotion=='분노','Emotion'] = 3
data.loc[data.Emotion=='혐오','Emotion'] = 3


pos_strong = data.loc[data['Emotion'] == 0]
pos_week = data.loc[data['Emotion'] == 1]
neg_strong = data.loc[data['Emotion'] == 3]
neg_week = data.loc[data['Emotion'] == 2]


# Train : Validation = 8 : 2
pos_strong_train =  pos_strong.iloc[:3865]
pos_strong_test = pos_strong.iloc[3866:4831]


pos_week_train =  pos_week.iloc[:3865]
pos_week_test = pos_week.iloc[3865:]


neg_strong_train =  neg_strong.iloc[:3865]
neg_strong_test = neg_strong.iloc[3866:4831]

neg_week_train =  neg_week.iloc[:3865]
neg_week_test = neg_week.iloc[3866:4831]

# Concatenate
data_train = pd.concat([pos_strong_train,pos_week_train,neg_strong_train,neg_week_train])
data_test = pd.concat([pos_strong_test,pos_week_test,neg_strong_test,neg_week_test])

#Shuffle
dataset_train = data_train.sample(frac=1)
dataset_test = data_test.sample(frac=1)

dataset_train.to_excel('./drive/MyDrive/21-2/data_train.xlsx', index = False) 
dataset_test.to_excel('./drive/MyDrive/21-2/data_test.xlsx', index = False)

a = data_train['Emotion'].to_list()

print("강한 긍정 : ",a.count(0))
print("약한 긍정 : ",a.count(1))
print("약한 부정 : ",a.count(2))
print("강한 부정 : ",a.count(3))

data_train = []
for q, label in zip(dataset_train['Sentence'], dataset_train['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_train.append(data)

data_test = []
for q, label in zip(dataset_test['Sentence'], dataset_test['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_test.append(data)



강한 긍정 :  3865
약한 긍정 :  3865
약한 부정 :  3865
강한 부정 :  3865


# **데이터 전처리 과정**

In [ ]:
import pandas as pd

data = pd.read_excel('./한국어_단발성_대화_데이터셋.xlsx') #경로 설정

# 문장내용과 감정 Column 만 추출
data = data[['Sentence','Emotion']]

# 강한 긍정 : 행복 (0)
# 약한 긍정 : 중립 (1)
# 약한 부정 : 공포, 놀람, 슬픔 (2)
# 강한 부정 : 혐오, 분노 (3)

data.loc[data.Emotion=='행복','Emotion'] = 0
data.loc[data.Emotion=='중립','Emotion'] = 1
data.loc[data.Emotion=='공포','Emotion'] = 2
data.loc[data.Emotion=='놀람','Emotion'] = 2
data.loc[data.Emotion=='슬픔','Emotion'] = 2
data.loc[data.Emotion=='분노','Emotion'] = 3
data.loc[data.Emotion=='혐오','Emotion'] = 3


pos_strong = data.loc[data['Emotion'] == 0]
pos_week = data.loc[data['Emotion'] == 1]
neg_strong = data.loc[data['Emotion'] == 3]
neg_week = data.loc[data['Emotion'] == 2]


# Train : Validation = 8 : 2
pos_strong_train =  pos_strong.iloc[:4831]
pos_strong_test = pos_strong.iloc[4831:]


pos_week_train =  pos_week.iloc[:3865]
pos_week_test = pos_week.iloc[3865:]

neg_strong_train =  neg_strong.iloc[:8875]
neg_strong_test = neg_strong.iloc[8875:]

neg_week_train =  neg_week.iloc[:13306]
neg_week_test = neg_week.iloc[13306:]

# Concatenate
data_train = pd.concat([pos_strong_train,pos_week_train,neg_strong_train,neg_week_train])
data_test = pd.concat([pos_strong_test,pos_week_test,neg_strong_test,neg_week_test])

#Shuffle
dataset_train = data_train.sample(frac=1)
dataset_test = data_test.sample(frac=1)

dataset_train.to_excel('./drive/MyDrive/21-2/data_train.xlsx', index = False) 
dataset_test.to_excel('./drive/MyDrive/21-2/data_test.xlsx', index = False)

a = data_train['Emotion'].to_list()

print("강한 긍정 : ",a.count(0))
print("약한 긍정 : ",a.count(1))
print("약한 부정 : ",a.count(2))
print("강한 부정 : ",a.count(3))

data_train = []
for q, label in zip(dataset_train['Sentence'], dataset_train['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_train.append(data)

data_test = []
for q, label in zip(dataset_test['Sentence'], dataset_test['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_test.append(data)



강한 긍정 :  4831
약한 긍정 :  3865
약한 부정 :  13306
강한 부정 :  8875


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

      
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 128
batch_size = 32
warmup_ratio = 0.1
num_epochs = 40
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(data_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(data_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5,shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('skt/kobert-base-v1')
        #self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# **Early Stopping**

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False):
        self.patience = patience
        self.verbose = True
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_min = np.Inf
        self.delta = 0
        self.path = './drive/MyDrive/21-2/check/chekpoint.pt'

    def __call__(self, val_acc, model):
        score =  val_acc

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
         
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0
            
    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation Accuracy  ({self.val_acc_min:.6f} --> {val_acc:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_acc_min = val_acc

In [ ]:
for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

writer.close()
!kill 462
%load_ext tensorboard
%tensorboard --logdir ./runs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
writer = SummaryWriter()

early_stopping = EarlyStopping(patience = 10, verbose = True)
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()

    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    writer.add_scalar('Accuracy/train',train_acc / (batch_id+1), e)
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
   
    writer.add_scalar('Accuracy/test',test_acc / (batch_id+1), e)
    early_stopping(test_acc / (batch_id+1), model)
    if early_stopping.early_stop:
        print("Early stopping")
        print(f'epoch {e+1} test acc {test_acc / (batch_id+1)}')
        break   
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
%load_ext tensorboard
%tensorboard --logdir ./runs

# **Load Check Point**

In [ ]:
model.load_state_dict(torch.load('./drive/MyDrive/21-2/check/chekpoint.pt'))

<All keys matched successfully>

# **Prediction**

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
                 
        print(predict_sentence)
        print("강한 긍정 : ",logits[0])
        print("약한 긍정 : ",logits[1])
        print("강한 부정 : ",logits[2])
        print("약한 부정 : ",logits[3])


In [ ]:
predict('증거 하나도 없이 변명만 내미는 너네! 살인자는 신변보호 붙이고. 피해자가 내미는 증거와. 의심 정황은 말살? 이게 공산국가냐?')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


증거 하나도 없이 변명만 내미는 너네! 살인자는 신변보호 붙이고. 피해자가 내미는 증거와. 의심 정황은 말살? 이게 공산국가냐?
강한 긍정 :  -2.141508
약한 긍정 :  -1.3850843
강한 부정 :  -1.7602284
약한 부정 :  5.0810475


In [ ]:
predict('사람 죽였는데 10년이라...재판부 전부 갈아치워야함.살인은 형량 최소 50년 부터 시작해야함!!')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


사람 죽였는데 10년이라...재판부 전부 갈아치워야함.살인은 형량 최소 50년 부터 시작해야함!!
강한 긍정 :  -2.8496068
약한 긍정 :  -1.3263507
강한 부정 :  -1.0053943
약한 부정 :  4.922746


In [ ]:
predict('쟤네 둘이 너무 행복해 보인다 ㅠㅠ')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


쟤네 둘이 너무 행복해 보인다 ㅠㅠ
강한 긍정 :  5.0546007
약한 긍정 :  -2.0384493
강한 부정 :  -0.44883215
약한 부정 :  -1.7718196


In [ ]:
predict('나 어떡해 ㅠㅠ')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


나 어떡해 ㅠㅠ
강한 긍정 :  -1.9261116
약한 긍정 :  -1.7733746
강한 부정 :  5.6075993
약한 부정 :  -1.2394574
